<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/save_timm_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install onnx-simplifier
!pip install timm

In [ ]:
import timm
import torch
model = timm.create_model('vit_tiny_patch16_224', pretrained=True)
model.eval()

dummy_input = torch.ones(1, 3, 224, 224, dtype=torch.float)
name = 'vit_tiny.onnx'
torch.onnx.export(model, dummy_input, name, verbose=True,opset_version=11)
!python3 -m onnxsim 'vit_tiny.onnx' 'vit_tiny_1.onnx'



In [ ]:
model = timm.create_model('deit_tiny_patch16_224', pretrained=True)
model.eval()

dummy_input = torch.ones(1, 3, 224, 224, dtype=torch.float)
name = 'deit_tiny.onnx'
torch.onnx.export(model, dummy_input, name, verbose=True,opset_version=11)
!python3 -m onnxsim 'deit_tiny.onnx' 'deit_tiny_1.onnx'

Downloading: "https://dl.fbaipublicfiles.com/deit/deit_tiny_patch16_224-a1311bcf.pth" to /root/.cache/torch/hub/checkpoints/deit_tiny_patch16_224-a1311bcf.pth
/usr/local/lib/python3.7/dist-packages/torch/__init__.py:676: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message
/usr/local/lib/python3.7/dist-packages/timm/models/vision_transformer.py:202: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  qkv = self.qkv(x).reshape(B, N

graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %cls_token : Float(1, 1, 192, strides=[192, 192, 1], requires_grad=1, device=cpu),
      %pos_embed : Float(1, 197, 192, strides=[37824, 192, 1], requires_grad=1, device=cpu),
      %patch_embed.proj.weight : Float(192, 3, 16, 16, strides=[768, 256, 16, 1], requires_grad=1, device=cpu),
      %patch_embed.proj.bias : Float(192, strides=[1], requires_grad=1, device=cpu),
      %blocks.0.norm1.weight : Float(192, strides=[1], requires_grad=1, device=cpu),
      %blocks.0.norm1.bias : Float(192, strides=[1], requires_grad=1, device=cpu),
      %blocks.0.attn.qkv.bias : Float(576, strides=[1], requires_grad=1, device=cpu),
      %blocks.0.attn.proj.bias : Float(192, strides=[1], requires_grad=1, device=cpu),
      %blocks.0.norm2.weight : Float(192, strides=[1], requires_grad=1, device=cpu),
      %blocks.0.norm2.bias : Float(192, strides=[1], requires_grad=1, device=cpu),
      %blo